In [14]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup

In [ ]:
holywood_df = pd.read_csv("highest-holywood-grossing-movies-with-poster-links(1).csv")
if "Poster Link" not in holywood_df.columns:
    holywood_df["Poster Link"] = ""
if "Unnamed: 0" in holywood_df.columns:
    holywood_df = holywood_df.drop(labels=["Unnamed: 0"], axis=1)
holywood_df.head()

,Title,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License,wikidata_id,Poster Link
0,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Action,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
1,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Adventure,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
2,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Fantasy,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
3,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Sci-Fi,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
4,Avengers: Endgame,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000.0,357115007.0,858373000,1941066100,2799439100,24-Apr-19,Action,181,PG-13,http://www.wikidata.org/entity/Q23781155,//upload.wikimedia.org/wikipedia/en/0/0d/Aveng...


In [16]:
for i in range(len(holywood_df)):
    if isinstance(holywood_df.loc[i, "Poster Link"], str):
        # print(holywood_df.loc[i, "Poster Link"])
        continue

    if i > 0 and holywood_df.loc[i, "Title"] == holywood_df.loc[i - 1, "Title"]:
        holywood_df.loc[i, "Poster Link"] = holywood_df.loc[i - 1, "Poster Link"]
        holywood_df.loc[i, "wikidata_id"] = holywood_df.loc[i - 1, "wikidata_id"]
        continue

    # title = holywood_df.loc[i, "Title"]
    # year = holywood_df.loc[i, "Year"]
    # title_query = title.replace(" ", "+")

    # URL_SEARCH = f"https://en.wikipedia.org/w/rest.php/v1/search/title?q={title_query}&limit=30"
    # req_search = requests.get(URL_SEARCH)
    # search_result = json.loads(req_search.content)["pages"]
    
    # URL_PAGE = None
    # for page in search_result:
    #     if "film" in page["title"].lower() and str(year) in page["description"].lower():
    #         URL_PAGE = "https://en.wikipedia.org/wiki/" + page["key"]
    #         break
    wikidata_uri = holywood_df.loc[i, "wikidata_id"]

    if not isinstance(wikidata_uri, str):
        continue

    # try:
    if wikidata_uri.startswith("https://www.wikidata.org/wiki/"):
        wikidata_uri = wikidata_uri.replace("https://www.wikidata.org/wiki/", "http://www.wikidata.org/entity/")
        holywood_df.loc[i, "wikidata_id"] = wikidata_uri

    print(wikidata_uri)
    
    wikidata_id = wikidata_uri.removeprefix("http://www.wikidata.org/entity/")
    req_wikidata = requests.get(wikidata_uri)
    res = json.loads(req_wikidata.content)
    if "enwiki" not in res["entities"][wikidata_id]["sitelinks"]:
        for wikilink in res["entities"][wikidata_id]["sitelinks"]:
            URL_PAGE = res["entities"][wikidata_id]["sitelinks"][wikilink]["url"]
            req_page = requests.get(URL_PAGE)
            soup = BeautifulSoup(req_page.content, 'html5lib')
            table = soup.find("table")
            if table == None:
                continue
            
            img = table.find("img")
            if img == None:
                continue

            img = img.get("src")
            holywood_df.loc[i, "Poster Link"] = img
            print(f"{i} {wikilink}: {img}")
            break
    else:
        URL_PAGE = res["entities"][wikidata_id]["sitelinks"]["enwiki"]["url"]
        req_page = requests.get(URL_PAGE)
        soup = BeautifulSoup(req_page.content, 'html5lib')
        table = soup.find("table")
        img = table.find("img").get("src")
        holywood_df.loc[i, "Poster Link"] = img
        print(f"{i} {img}")
    # except:
    #     continue

http://www.wikidata.org/entity/Q27044293
26 //upload.wikimedia.org/wikipedia/en/9/9d/Disney_The_Lion_King_2019.jpg
http://www.wikidata.org/entity/Q24733929
38 //upload.wikimedia.org/wikipedia/en/thumb/8/89/Frozen_II_%282019_animated_film%29.jpg/220px-Frozen_II_%282019_animated_film%29.jpg
http://www.wikidata.org/entity/Q108671102
55 //upload.wikimedia.org/wikipedia/en/thumb/4/44/The_Super_Mario_Bros._Movie_poster.jpg/220px-The_Super_Mario_Bros._Movie_poster.jpg
http://www.wikidata.org/entity/Q232009
61 //upload.wikimedia.org/wikipedia/en/d/df/Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_2.jpg
http://www.wikidata.org/entity/Q18486020
http://www.wikidata.org/entity/Q20977110
130 //upload.wikimedia.org/wikipedia/en/a/af/Star_Wars_The_Rise_of_Skywalker_poster.jpg
http://www.wikidata.org/entity/Q161678
204 //upload.wikimedia.org/wikipedia/en/thumb/2/2d/Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_1.jpg/220px-Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_1.jpg
http://www.

In [17]:
holywood_df

,Title,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License,wikidata_id,Poster Link
0,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Action,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
1,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Adventure,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
2,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Fantasy,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
3,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000.0,77025481.0,785221649,2138484377,2923706026,16-Dec-09,Sci-Fi,162,PG-13,http://www.wikidata.org/entity/Q24871,//upload.wikimedia.org/wikipedia/en/thumb/d/d6...
4,Avengers: Endgame,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000.0,357115007.0,858373000,1941066100,2799439100,24-Apr-19,Action,181,PG-13,http://www.wikidata.org/entity/Q23781155,//upload.wikimedia.org/wikipedia/en/0/0d/Aveng...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,Deja Vu,"After a ferry is bombed in New Orleans, an A.T...",2006,Walt Disney Studios Motion Pictures,75000000.0,20574802.0,64038616,116518934,180557550,22-Nov-06,Thriller,126,PG-13,http://www.wikidata.org/entity/Q252409,//upload.wikimedia.org/wikipedia/en/thumb/c/cf...
3594,M3GAN,A robotics engineer at a toy company builds a ...,2022,Universal Pictures,NaN,30429860.0,95043350,84925523,179968873,22-Nov-06,Action,126,PG-13,http://www.wikidata.org/entity/Q110661062,//upload.wikimedia.org/wikipedia/en/0/03/M3GAN...
3595,M3GAN,A robotics engineer at a toy company builds a ...,2022,Universal Pictures,NaN,30429860.0,95043350,84925523,179968873,22-Nov-06,Crime,126,PG-13,http://www.wikidata.org/entity/Q110661062,//upload.wikimedia.org/wikipedia/en/0/03/M3GAN...
3596,M3GAN,A robotics engineer at a toy company builds a ...,2022,Universal Pictures,NaN,30429860.0,95043350,84925523,179968873,22-Nov-06,Sci-Fi,126,PG-13,http://www.wikidata.org/entity/Q110661062,//upload.wikimedia.org/wikipedia/en/0/03/M3GAN...


In [18]:
holywood_df.to_csv("highest-holywood-grossing-movies-with-poster-links.csv")